In [1]:
import argparse
import csv
import os

import numpy as np
import torch
from torch.autograd import Variable
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets

import models
from utils import progress_bar
from train import myModel
from cutout import *


In [3]:
cutout=False
batch_size=128
length=16


if cutout:
    
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465),
                             (0.2023, 0.1994, 0.2010)),
            ])
    transform_train.transforms.append(Cutout(n_holes=1, length=length))
else:
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465),
                             (0.2023, 0.1994, 0.2010)),
            ])


transform_test = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
        ])

trainset = datasets.CIFAR10(root='~/data', train=True, download=False,
                            transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset,
                                          batch_size=batch_size,
                                          shuffle=True, num_workers=8)

testset = datasets.CIFAR10(root='~/data', train=False, download=False,
                           transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100,
                                         shuffle=False, num_workers=8)

In [4]:
model_list=["ResNet18","ResNet34","ResNet50"]
name_list=["ResNet18_alpha=1","ResNet34_alpha=1","ResNet50_alpha=1"]
Tr_acclist_alpha=[]
Te_acclist_alpha=[]

for i in range(3):
    sname=name_list[i]
    mod=model_list[i]
    resnet=myModel(mod,sname,con=False)
    print("start at ",resnet.start_epoch)
    print("best = ",resnet.best_acc)
    tr_acc=[]
    te_acc=[]

    if not os.path.exists(resnet.logname):
        with open(resnet.logname, 'w') as logfile:
            logwriter = csv.writer(logfile, delimiter=',')
            logwriter.writerow(['epoch', 'train loss', 'reg loss', 'train acc',
                            'test loss', 'test acc'])    
    
    for epoch in range(resnet.start_epoch, 100):
        train_loss, reg_loss, train_acc = resnet.train(epoch,trainloader,mixup=True,alpha=1.)
        print("Epoch %d Train acc %f percent "%(epoch,train_acc))
        tr_acc.append(train_acc)
        test_loss, test_acc = resnet.test(epoch,testloader)
        print("Epoch %d Test acc %f percent "%(epoch,test_acc))
        te_acc.append(test_acc)
        resnet.adjust_learning_rate(resnet.optimizer, epoch)
        with open(resnet.logname, 'a') as logfile:
            logwriter = csv.writer(logfile, delimiter=',')
            logwriter.writerow([epoch, train_loss, reg_loss, float(train_acc.data), test_loss,
                            float(test_acc.data)])
    print("best = ",resnet.best_acc)
    train=[]
    test=[]
    for i in range(len(tr_acc)):
        train.append(float(tr_acc[i].data))
        test.append(float(te_acc[i].data))
    print(sname+" train")
    print(train)
    print(sname+" test")
    print(test)
    Tr_acclist_alpha.append(train)
    Te_acclist_alpha.append(test)


==> Building model..
5
Using CUDA..
start at  0
best =  0

Epoch: 0
Epoch 0 Train acc 16.341999 percent 
Saving..
Epoch 0 Test acc 21.440001 percent 
0.01

Epoch: 1
Epoch 1 Train acc 24.954000 percent 
Saving..
Epoch 1 Test acc 26.309999 percent 

Epoch: 2
Epoch 2 Train acc 28.172001 percent 
Saving..
Epoch 2 Test acc 29.600000 percent 

Epoch: 3
Epoch 3 Train acc 30.910000 percent 
Saving..
Epoch 3 Test acc 34.770000 percent 

Epoch: 4
Epoch 4 Train acc 33.924000 percent 
Saving..
Epoch 4 Test acc 37.020000 percent 

Epoch: 5
Epoch 5 Train acc 36.686001 percent 
Saving..
Epoch 5 Test acc 39.790001 percent 

Epoch: 6
Epoch 6 Train acc 39.824001 percent 
Epoch 6 Test acc 39.139999 percent 

Epoch: 7
Epoch 7 Train acc 41.161999 percent 
Saving..
Epoch 7 Test acc 42.610001 percent 

Epoch: 8
Epoch 8 Train acc 43.220001 percent 
Saving..
Epoch 8 Test acc 43.070000 percent 

Epoch: 9
Epoch 9 Train acc 45.453999 percent 
Saving..
Epoch 9 Test acc 46.020000 percent 

Epoch: 10
Epoch 10 Train 

KeyboardInterrupt: 

In [4]:
print("best = ",resnet.best_acc)

best =  tensor(87.9000)


In [2]:
cutout=True
batch_size=128
length=16


if cutout:
    
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465),
                             (0.2023, 0.1994, 0.2010)),
            ])
    transform_train.transforms.append(Cutout(n_holes=1, length=length))
else:
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465),
                             (0.2023, 0.1994, 0.2010)),
            ])


transform_test = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
        ])

trainset = datasets.CIFAR10(root='~/data', train=True, download=False,
                            transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset,
                                          batch_size=batch_size,
                                          shuffle=True, num_workers=8)

testset = datasets.CIFAR10(root='~/data', train=False, download=False,
                           transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100,
                                         shuffle=False, num_workers=8)



model_list=["ResNet18","ResNet34","ResNet50"]
name_list=["ResNet18_length=16","ResNet34_length=16","ResNet50_length=16"]
Tr_acclist_length=[]
Te_acclist_length=[]

for i in range(3):
    sname=name_list[i]
    mod=model_list[i]
    resnet=myModel(mod,sname,con=False)
    print("start at ",resnet.start_epoch)
    print("best = ",resnet.best_acc)
    tr_acc=[]
    te_acc=[]

    if not os.path.exists(resnet.logname):
        with open(resnet.logname, 'w') as logfile:
            logwriter = csv.writer(logfile, delimiter=',')
            logwriter.writerow(['epoch', 'train loss', 'reg loss', 'train acc',
                            'test loss', 'test acc'])    
    
    for epoch in range(resnet.start_epoch, 100):
        train_loss, reg_loss, train_acc = resnet.train(epoch,trainloader,mixup=False)
        print("Epoch %d Train acc %f percent "%(epoch,train_acc))
        tr_acc.append(train_acc)
        test_loss, test_acc = resnet.test(epoch,testloader)
        print("Epoch %d Test acc %f percent "%(epoch,test_acc))
        te_acc.append(test_acc)
        resnet.adjust_learning_rate(resnet.optimizer, epoch)
        with open(resnet.logname, 'a') as logfile:
            logwriter = csv.writer(logfile, delimiter=',')
            logwriter.writerow([epoch, train_loss, reg_loss, float(train_acc.data), test_loss,
                            float(test_acc.data)])
    print("best = ",resnet.best_acc)
    train=[]
    test=[]
    for i in range(len(tr_acc)):
        train.append(float(tr_acc[i].data))
        test.append(float(te_acc[i].data))
    print(sname+" train")
    print(train)
    print(sname+" test")
    print(test)
    Tr_acclist_length.append(train)
    Te_acclist_length.append(test)


NameError: name 'Cutout' is not defined